# LSTM для определения тематики книги по названию

one-to-all

In [ ]:
import json
import pandas as pd
import numpy as np

In [2]:
import nltk
import re
from nltk.corpus import words, stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer, ISRIStemmer, LancasterStemmer, PorterStemmer, RegexpStemmer, RSLPStemmer, SnowballStemmer
from nltk.corpus import words as nltk_words

In [3]:
from sklearn.model_selection import train_test_split 
from sklearn.metrics import *
from sklearn.utils import resample # будем делать downsumpling


In [4]:
from gensim.models import Word2Vec, KeyedVectors
#w2v_model = Word2Vec.load('enwv\en.model')
w2v_model = KeyedVectors.load_word2vec_format('enwv\GoogleNews-vectors-negative300.bin', binary= True)

C:\Users\blacat\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [5]:
import collections

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Embedding, Dropout, Conv1D, Dense, Flatten, LSTM, Bidirectional
from keras.layers.pooling import GlobalMaxPool1D, MaxPooling1D, MaxPool1D
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint

C:\Users\blacat\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
Couldn't import dot_parser, loading of dot files will not be possible.


In [6]:
#расширим список стоп-слов
## встречаются буквы по отдельности (окочания после апострофа и тп), возможно ошибки написания
mystopwords = set(stopwords.words('english') + ['a', 'b', 'c', 'd', 'e', 'f', 'j', 'h', 
           'g', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 
           's', 't', 'u', 'v', 'w', 'x', 'y', 'z']) 

eng_let = ['a', 'b', 'c', 'd', 'e', 'f', 'j', 'h', 'i', 
           'g', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 
           's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ']

In [7]:
lemm = WordNetLemmatizer()
def tokenize(x):
    article = re.compile(u'[a-z]+')    
    return article.findall(x)

def stop_words(x):
    return [token for token in x if not token in mystopwords]

def lemma(x):
    return [lemm.lemmatize(token) for token in x ]

In [8]:
books = pd.read_csv('title.csv')

C:\Users\blacat\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (2,5,7,9,11,13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
allbooks = len(books)
allbooks

58751

In [10]:
subj_list = ['children', 'fantasy','medicine', 'music','mystery and detective stories', 
             'recipes', 'religion', 'romance', 'science','science fiction']

In [11]:
# число книг с определенной темой
b_num = {}
for i in subj_list:
    books[i].sum()
    b_num[i] = books[i].sum()
b_num

#subj_sum = books.religion.sum()
#subj_sum

{'children': 4235.0,
 'fantasy': 4384.0,
 'medicine': 3144.0,
 'music': 6077.0,
 'mystery and detective stories': 2316.0,
 'recipes': 1225.0,
 'religion': 11098.0,
 'romance': 10181.0,
 'science': 13222.0,
 'science fiction': 3770.0}

создадим сбалансированную выборку:


In [12]:
#В словарях будем хранить информацию для всех классов;
Xtr_subj = {}
Xtst_subj = {}
ytr_subj = {}
ytst_subj = {}
vocab_sz_subj = {}
ebmedd_wts_subj = {}
maxlen_subj = {}


for subj in subj_list:
    print(subj)
    df_1 = books[['title','words',subj]][books[subj] == 1]
    df_majority = books[['title','words',subj]][books[subj] != 1]
    
    # Downsample majority class
    df_0 = resample(df_majority,replace=False,    # sample without replacement
                                 n_samples=int(b_num[subj]),     # to match minority class
                                 random_state=888) # reproducible results
    df_downsampl = pd.concat([df_0, df_1])
    df_downsampl.reset_index(inplace = True)
    del df_downsampl['index']
    X = df_downsampl['words']
    y = df_downsampl[subj]
    
    print(subj,'max sumples: ' + str(df_downsampl.shape))
    X_train, X_test, ytr, ytst = train_test_split(X, y, test_size=0.2, random_state=888, shuffle = True)
    
    ytr_bin = to_categorical(ytr)
    ytst_bin = to_categorical(ytst)
    ytr_subj[subj] = ytr_bin
    ytst_subj[subj] = ytst_bin
    
    counter = collections.Counter()
    maxlen = 0    
    for key, value in df_downsampl.title.items():
        title_words = stop_words(lemma(tokenize(value)))
        len_title = len(title_words)
        for w in title_words:
            counter[w] += 1
        if len_title>maxlen:
            maxlen = len_title
    maxlen_subj[subj] = maxlen
    all_w = len(counter) # число всех слов в названиях книг
    print('All words: ' + str(all_w) ,'. Max title length: '+ str(maxlen))
    
    VOCAB_SIZE = all_w #используем все слова для создания эмбеддингов для лучшего результата. 
    EMBED_SIZE = 300
    NUM_FILTERS = 256
    NUM_WORDS = 5
    BATCH_SIZE = 128
    NUM_EPOCHS = 20
    
    word2index = collections.defaultdict(int)
    for wid, word in enumerate(counter.most_common(VOCAB_SIZE)):
        word2index[word[0]] = wid
        vocab_sz = len(word2index) 
        index2word = {v:k for k, v in word2index.items()}
        
    vocab_sz_subj[subj] = vocab_sz
    print('Vocabular size: '+ str(vocab_sz))
    
    embedd_wts = np.zeros((vocab_sz, EMBED_SIZE))
    for word, index in word2index.items():
        #print(word, index)
        try:
            embedd_wts[index, :] = w2v_model[word]
        except KeyError:
            pass
    ebmedd_wts_subj[subj] = embedd_wts
    
    Xtrain, Xtest = [], []
    for i in X_train:
        title_words = stop_words(lemma(tokenize(i)))
        wids = [word2index[word] for word in title_words]
        Xtrain.append(wids)
    
    for i in X_test:
        title_words = stop_words(lemma(tokenize(i)))
        wids = [word2index[word] for word in title_words]
        Xtest.append(wids)
        
    Xtr = np.array(pad_sequences(Xtrain , maxlen = maxlen))
    Xtst = np.array(pad_sequences(Xtest, maxlen = maxlen)) 
    Xtr_subj[subj] = Xtr
    Xtst_subj[subj] = Xtst
    print();

children
children max sumples: (8470, 3)
All words: 8379 . Max title length: 21
Vocabular size: 8379

fantasy
fantasy max sumples: (8768, 3)
All words: 7960 . Max title length: 22
Vocabular size: 7960

medicine
medicine max sumples: (6288, 3)
All words: 7240 . Max title length: 41
Vocabular size: 7240

music
music max sumples: (12154, 3)
All words: 10888 . Max title length: 22
Vocabular size: 10888

mystery and detective stories
mystery and detective stories max sumples: (4632, 3)
All words: 5395 . Max title length: 22
Vocabular size: 5395

recipes
recipes max sumples: (2450, 3)
All words: 3204 . Max title length: 22
Vocabular size: 3204

religion
religion max sumples: (22196, 3)
All words: 15585 . Max title length: 26
Vocabular size: 15585

romance
romance max sumples: (20362, 3)
All words: 12737 . Max title length: 22
Vocabular size: 12737

science
science max sumples: (26444, 3)
All words: 17139 . Max title length: 26
Vocabular size: 17139

science fiction
science fiction max sumple

In [13]:
maxlen_subj

{'children': 21,
 'fantasy': 22,
 'medicine': 41,
 'music': 22,
 'mystery and detective stories': 22,
 'recipes': 22,
 'religion': 26,
 'romance': 22,
 'science': 26,
 'science fiction': 26}

# Реализация нейронной сети

LSTM

In [14]:
def title_model(vocab_sz, EMBED_SIZE, maxlen, embedd_wts):
    md = Sequential()
    md.add(Embedding(vocab_sz, EMBED_SIZE, input_length=maxlen, weights = [embedd_wts], trainable=False))
    md.add(LSTM(256, input_shape=(vocab_sz, EMBED_SIZE), return_sequences=True , activation= 'relu' ))
    md.add(Dropout(0.15))
    md.add(LSTM(128 , return_sequences=True , activation= 'relu' ))
    md.add(Dropout(0.15))
    md.add(LSTM(64, activation= 'relu' ))
    md.add(Dropout(0.15))
    md.add(Dense(2, activation= 'softmax'))
    optimizer = RMSprop(1e-4)
    md.compile(optimizer= optimizer, loss = 'categorical_crossentropy', metrics=['accuracy'])
    return md

In [20]:
history_subj = {} # словарь сохранить историю вывода модели
predict_subj = {} # словарь хранить предсказания

for subj in subj_list: 
    print(subj, vocab_sz_subj[subj], maxlen_subj[subj])
    
    checkp = ModelCheckpoint(filepath='title_weights\weights.best.title.'+ subj +'.hdf5', 
                               verbose=1, save_best_only=True)
    subj_process = title_model(vocab_sz_subj[subj], EMBED_SIZE, maxlen_subj[subj], \
                               ebmedd_wts_subj[subj]).fit(Xtr_subj[subj], ytr_subj[subj], 
                                                          batch_size = 128,
                                                          epochs=15, validation_split=0.15, 
                                                          callbacks=[checkp])
    history_subj[subj] = subj_process.history
    predict_subj[subj] = subj_process.model.predict_classes(Xtst_subj[subj], batch_size = BATCH_SIZE, verbose = 1)
    print();

children 8379 21
Train on 5759 samples, validate on 1017 samples
Epoch 1/15
5759/5759 [==============================] - 27s 5ms/step - loss: 0.6924 - acc: 0.5208 - val_loss: 0.6923 - val_acc: 0.5064
Epoch 2/15
5759/5759 [==============================] - 22s 4ms/step - loss: 0.6870 - acc: 0.5503 - val_loss: 0.6797 - val_acc: 0.5792
Epoch 3/15
5759/5759 [==============================] - 22s 4ms/step - loss: 0.6748 - acc: 0.5666 - val_loss: 0.6560 - val_acc: 0.5969
Epoch 4/15
5759/5759 [==============================] - 22s 4ms/step - loss: 0.6495 - acc: 0.6103 - val_loss: 0.6428 - val_acc: 0.5998
Epoch 5/15
5759/5759 [==============================] - 22s 4ms/step - loss: 0.6213 - acc: 0.6506 - val_loss: 0.6036 - val_acc: 0.6676
Epoch 6/15
5759/5759 [==============================] - 23s 4ms/step - loss: 0.5941 - acc: 0.6762 - val_loss: 0.6452 - val_acc: 0.6401
Epoch 7/15
5759/5759 [==============================] - 22s 4ms/step - loss: 0.5841 - acc: 0.6812 - val_loss: 0.5939 - val_ac

Epoch 13/15
5961/5961 [==============================] - 25s 4ms/step - loss: 0.4955 - acc: 0.7494 - val_loss: 0.4871 - val_acc: 0.7711
Epoch 14/15
5961/5961 [==============================] - 27s 4ms/step - loss: 0.4904 - acc: 0.7568 - val_loss: 0.5026 - val_acc: 0.7455
Epoch 15/15
1754/1754 [==============================] - 3s 2ms/step

medicine 7240 41
Train on 4275 samples, validate on 755 samples
Epoch 1/15
4275/4275 [==============================] - 39s 9ms/step - loss: 0.6918 - acc: 0.5331 - val_loss: 0.6893 - val_acc: 0.6119
Epoch 2/15
4275/4275 [==============================] - 37s 9ms/step - loss: 0.6874 - acc: 0.5808 - val_loss: 0.7011 - val_acc: 0.4781
Epoch 3/15
4275/4275 [==============================] - 34s 8ms/step - loss: 0.6797 - acc: 0.6058 - val_loss: 0.6791 - val_acc: 0.4861
Epoch 4/15
4275/4275 [==============================] - 35s 8ms/step - loss: 0.6616 - acc: 0.6239 - val_loss: 0.6365 - val_acc: 0.6808
Epoch 5/15
4275/4275 [==============================] 

Epoch 9/15
8264/8264 [==============================] - 39s 5ms/step - loss: 0.4608 - acc: 0.7773 - val_loss: 0.4904 - val_acc: 0.7567
Epoch 10/15
8264/8264 [==============================] - 38s 5ms/step - loss: 0.4564 - acc: 0.7806 - val_loss: 0.4210 - val_acc: 0.7971
Epoch 11/15
8264/8264 [==============================] - 40s 5ms/step - loss: 0.4461 - acc: 0.7865 - val_loss: 0.4754 - val_acc: 0.8053
Epoch 12/15
8264/8264 [==============================] - 39s 5ms/step - loss: 0.4295 - acc: 0.7951 - val_loss: 0.4076 - val_acc: 0.8101
Epoch 13/15
8264/8264 [==============================] - 39s 5ms/step - loss: 0.4208 - acc: 0.8053 - val_loss: 0.4323 - val_acc: 0.8040
Epoch 14/15
8264/8264 [==============================] - 38s 5ms/step - loss: 0.4098 - acc: 0.8054 - val_loss: 0.4079 - val_acc: 0.8129
Epoch 15/15
2431/2431 [==============================] - 5s 2ms/step

mystery and detective stories 5395 22
Train on 3149 samples, validate on 556 samples
Epoch 1/15
3149/3149 [========

Epoch 5/15
1666/1666 [==============================] - 8s 5ms/step - loss: 0.6515 - acc: 0.6465 - val_loss: 0.6188 - val_acc: 0.7857
Epoch 6/15
1666/1666 [==============================] - 8s 5ms/step - loss: 0.6374 - acc: 0.6567 - val_loss: 0.5802 - val_acc: 0.6939
Epoch 7/15
1666/1666 [==============================] - 8s 5ms/step - loss: 0.5986 - acc: 0.6837 - val_loss: 0.9098 - val_acc: 0.5952
Epoch 8/15
1666/1666 [==============================] - 8s 5ms/step - loss: 0.6069 - acc: 0.6951 - val_loss: 0.7318 - val_acc: 0.5442
Epoch 9/15
1666/1666 [==============================] - 8s 5ms/step - loss: 0.6170 - acc: 0.6981 - val_loss: 0.6605 - val_acc: 0.6020
Epoch 10/15
1666/1666 [==============================] - 8s 5ms/step - loss: 0.5534 - acc: 0.7209 - val_loss: 0.6209 - val_acc: 0.5476
Epoch 11/15
1666/1666 [==============================] - 9s 5ms/step - loss: 0.5314 - acc: 0.7509 - val_loss: 0.6216 - val_acc: 0.6224
Epoch 12/15
1666/1666 [==============================] - 9s 

Epoch 2/15
13845/13845 [==============================] - 80s 6ms/step - loss: 0.6068 - acc: 0.6626 - val_loss: 0.5558 - val_acc: 0.7340
Epoch 3/15
13845/13845 [==============================] - 77s 6ms/step - loss: 0.5532 - acc: 0.7112 - val_loss: 0.6806 - val_acc: 0.7148
Epoch 4/15
13845/13845 [==============================] - 81s 6ms/step - loss: 0.5332 - acc: 0.7397 - val_loss: 0.4749 - val_acc: 0.7754
Epoch 5/15
13845/13845 [==============================] - 79s 6ms/step - loss: 0.4788 - acc: 0.7620 - val_loss: 0.5390 - val_acc: 0.6984
Epoch 6/15
13845/13845 [==============================] - 79s 6ms/step - loss: 0.4713 - acc: 0.7706 - val_loss: 0.4422 - val_acc: 0.7917
Epoch 7/15
13845/13845 [==============================] - 82s 6ms/step - loss: 0.4337 - acc: 0.7894 - val_loss: 0.5430 - val_acc: 0.7340
Epoch 8/15
13845/13845 [==============================] - 80s 6ms/step - loss: 0.4292 - acc: 0.7931 - val_loss: 0.4087 - val_acc: 0.8028
Epoch 9/15
13845/13845 [=================

Epoch 14/15
17981/17981 [==============================] - 126s 7ms/step - loss: 0.3224 - acc: 0.8594 - val_loss: 0.3677 - val_acc: 0.8362
Epoch 15/15
5289/5289 [==============================] - 18s 3ms/step

science fiction 7248 26
Train on 5127 samples, validate on 905 samples
Epoch 1/15
5127/5127 [==============================] - 44s 9ms/step - loss: 0.6887 - acc: 0.5520 - val_loss: 0.6809 - val_acc: 0.5757
Epoch 2/15
5127/5127 [==============================] - 36s 7ms/step - loss: 0.6691 - acc: 0.6160 - val_loss: 0.6712 - val_acc: 0.5470
Epoch 3/15
5127/5127 [==============================] - 35s 7ms/step - loss: 0.6529 - acc: 0.6218 - val_loss: 0.6845 - val_acc: 0.5547
Epoch 4/15
5127/5127 [==============================] - 39s 8ms/step - loss: 0.6406 - acc: 0.6351 - val_loss: 0.6476 - val_acc: 0.6254
Epoch 5/15
5127/5127 [==============================] - 39s 8ms/step - loss: 0.6336 - acc: 0.6372 - val_loss: 0.8036 - val_acc: 0.5193
Epoch 6/15
5127/5127 [======================

In [21]:
ytst_subj[subj][:,1]

array([1., 1., 0., ..., 0., 1., 1.])

In [22]:
for subj in subj_list:
    print(subj)
    report = classification_report(ytst_subj[subj][:,1],predict_subj[subj])
    print(report)

children
             precision    recall  f1-score   support

        0.0       0.78      0.71      0.74       864
        1.0       0.72      0.80      0.76       830

avg / total       0.75      0.75      0.75      1694

fantasy
             precision    recall  f1-score   support

        0.0       0.84      0.67      0.74       872
        1.0       0.73      0.87      0.79       882

avg / total       0.78      0.77      0.77      1754

medicine
             precision    recall  f1-score   support

        0.0       0.78      0.83      0.81       638
        1.0       0.82      0.76      0.79       620

avg / total       0.80      0.80      0.80      1258

music
             precision    recall  f1-score   support

        0.0       0.86      0.75      0.80      1200
        1.0       0.78      0.88      0.83      1231

avg / total       0.82      0.82      0.82      2431

mystery and detective stories
             precision    recall  f1-score   support

        0.0       0.97  